### Reinforcement Learning on Markov Decision 

Import libraries of Python

In [1]:
import numpy as np
from anytree import Node
import pandas as pd

>Create class for the MDP model states

In [2]:
class State():
    def __init__(self, name, actions, next_states, probs, rewards):
        self.name = name
        self.actions = actions
        self.next_states = next_states
        self.probs = probs
        self.rewards = rewards
        if len(self.actions) == 0:
            self.isendstate = True
        else:
            self.isendstate = False

>Creating the data structure of the MDP model:

In [3]:
ending = State('11a', [], [], [], [])
TU10a = State('TU10a', ['P', 'R', 'S'], [ending, ending, ending], [1., 1., 1.], [-1., -1., -1.])
RU10a = State('RU10a', ['P', 'R', 'S'], [ending, ending, ending], [1., 1., 1.], [0., 0., 0.])
RD10a = State('RD10a', ['P', 'R', 'S'], [ending, ending, ending], [1., 1., 1.], [4., 4., 4.])
TD10a = State('TD10a', ['P', 'R', 'S'], [ending, ending, ending], [1., 1., 1.], [3., 3., 3.])
RU8a = State('RU8a', ['P', 'R', 'S'], [TU10a, RU10a, RD10a], [1., 1., 1.], [2., 0., -1.])
RD8a = State('RD8a', ['P', 'R'], [TD10a, RD10a], [1., 1.], [2., 0.])
TU10p = State('TU10p', ['P', 'R'], [RU10a, RU8a], [1., 1.], [2., 0.])
RU10p = State('RU10p', ['P', 'P', 'R', 'S'], [RU8a, RU10a, RU8a, RD8a], [0.5, 0.5, 1., 1.], [2., 2., 0., -1.])
RD10p = State('RD10p', ['P', 'P', 'R'], [RD8a, RD10a, RD8a], [0.5, 0.5, 1.], [2., 2., 0.])
RU8p = State('RU8p', ['P', 'R', 'S'], [TU10p, RU10p, RD10p], [1., 1., 1.], [2., 0., -1.])

states = [RU8p, TU10p, RU10p, RD10p, RU8a, RD8a, TU10a, RU10a, RD10a, TD10a, ending]

>The functions below prints the state properties: 

In [4]:
def get_state_names(states):
    State_list = []
    for i in states:
        State_list.append(i.name)
    return State_list

def get_policy_actions_or_values(Policy): #or Vinit
    Action_list = []
    for i in Policy:
        Action_list.append(Policy[i])
    return Action_list

## Codes for 3a:

> The following part prints out the ordered sequence of states/actions/rewards for 50 episodes:<br>
to be noted that the average return for 50 episodes vary in each run.

In [25]:
episodes = 50
totsum = 0
table = [['#Episode', 'States', 'Actions', 'Rewards', 'Return']]
for i in range(episodes):
    run = True
    line = []
    state_list = []
    act_list = []
    rew_list = []
    state = RU8p
    while (run):
        if state.isendstate == False:
            nAct = len(state.actions)  # len(ending.actions) = 0
            state_list.append(state.name)
            choose = np.int(np.random.randint(0, nAct, 1))
            act_list.append(state.actions[choose])
            rew_list.append(state.rewards[choose])
            state = state.next_states[choose]
        else:
            run = False
    line.append([i+1,state_list,act_list,rew_list,np.sum(rew_list)])
    table = np.vstack((table,line))
    totsum += np.sum(rew_list)
df = pd.DataFrame(table)
print(df)
# export_CSV =df.to_csv(r'C:\Users\ranab\OneDrive\PycharmProjects\RL_assignment\Episode_table.csv')
# print(pd.DataFrame(table))
print("Average return for the {} episodes is: {}".format(episodes, totsum / 50))

           0                           1             2                      3  \
0   #Episode                      States       Actions                Rewards   
1          1  [RU8p, TU10p, RU8a, RD10a]  [P, R, S, P]  [2.0, 0.0, -1.0, 4.0]   
2          2  [RU8p, TU10p, RU8a, RU10a]  [P, R, R, R]   [2.0, 0.0, 0.0, 0.0]   
3          3  [RU8p, TU10p, RU8a, RD10a]  [P, R, S, S]  [2.0, 0.0, -1.0, 4.0]   
4          4  [RU8p, RD10p, RD8a, TD10a]  [S, P, P, R]  [-1.0, 2.0, 2.0, 3.0]   
5          5  [RU8p, RU10p, RU8a, TU10a]  [R, R, P, R]  [0.0, 0.0, 2.0, -1.0]   
6          6        [RU8p, TU10p, RU10a]     [P, P, P]        [2.0, 2.0, 0.0]   
7          7        [RU8p, RD10p, RD10a]     [S, P, R]       [-1.0, 2.0, 4.0]   
8          8  [RU8p, TU10p, RU8a, RD10a]  [P, R, S, P]  [2.0, 0.0, -1.0, 4.0]   
9          9  [RU8p, RD10p, RD8a, TD10a]  [S, R, P, R]  [-1.0, 0.0, 2.0, 3.0]   
10        10  [RU8p, RD10p, RD8a, TD10a]  [S, P, P, P]  [-1.0, 2.0, 2.0, 3.0]   
11        11  [RU8p, RD10p, 

> The following function provides the state values 

In [14]:
def get_state_value(gamma,state):
    nAct = len(state.actions)
    state.val = np.zeros(nAct)
    uAct = len(set(state.actions)) #will remove duplicate values, just uniques
    if nAct == 0:
        state.val = 0.
    else:
        eq_prob = 1.0 / uAct

        for i in range(nAct):
            if state.name == '11a':
                nextVs = 0
            else:
                nextVs = get_state_value(gamma,state.next_states[i])
            state.val[i] = eq_prob * state.probs[i]*(state.rewards[i]+(gamma * nextVs))
    return np.sum(state.val)

In [15]:
i=0
row = [['No.','States','Values']]
for s in states:
    i+=1
    line = []
    value = get_state_value(1.0,s)
    line.append([i,s.name,value])
    row = np.vstack((row,line))
print(pd.DataFrame(row))
# df = pd.DataFrame(row)
# export_CSV=df.to_csv(r'C:\Users\ranab\OneDrive\PycharmProjects\RL_assignment\state_values.csv')

      0       1                   2
0   No.  States              Values
1     1    RU8p  3.5138888888888884
2     2   TU10p  1.6666666666666665
3     3   RU10p                 2.5
4     4   RD10p               5.375
5     5    RU8a  1.3333333333333333
6     6    RD8a                 4.5
7     7   TU10a                -1.0
8     8   RU10a                 0.0
9     9   RD10a                 4.0
10   10   TD10a                 3.0
11   11     11a                 0.0


## Codes for 3b:
> The following function evaluates the policy:

In [96]:
def Evaluation(Policy,Vinit,states):
    theta = 0.001
    gamma = 1.0
    cnd = True
    iter = 0
    table = np.vstack((get_state_names(states),get_policy_actions_or_values(Policy)))
    while(cnd):
        iter += 1
        lstAc=[]
        lstVal=[]
        delta = 0.0 #initially change in state value = 0
        for s in states:
            v = Vinit[s] #initially 0
            a = Policy[s] #P type:str
            sum = []
            for j in range(len(s.actions)):
                if s.actions[j] == a: #1p 2p
                    sum.append(s.probs[j]*(s.rewards[j]+gamma*Vinit[s.next_states[j]]))
            Vinit[s] = np.sum(sum)
            delta = max(delta,abs(v-Vinit[s]))
        table = np.vstack((table,get_policy_actions_or_values(Vinit)))
        cnd = (delta >= theta)
#     print("Number of iterations required in policy evaluation:", iter)
#     print(pd.DataFrame(table))
    df = pd.DataFrame(table)
    return Vinit,df

> Starting with the initial value arbitrarily from 0 for all the states. Also setting the initial policy to "Rock & Roll all night and Party every day" (i.e. policy should choose to party regardless of what state the agent is in)

In [110]:
Vinit = {RU8p:0, TU10p:0,RU10p:0,RD10p:0,RU8a:0,RD8a:0,TU10a:0,RU10a:0,RD10a:0,TD10a:0,ending:0}
Policy = {RU8p:'P', TU10p:'P',RU10p:'P',RD10p:'P',RU8a:'P',RD8a:'P',TU10a:'P',RU10a:'P',RD10a:'P',TD10a:'P',ending:'P'}

In [111]:
Vinit,df = Evaluation(Policy,Vinit,states)
print(df)

     0      1      2      3     4     5      6      7      8      9    10
0  RU8p  TU10p  RU10p  RD10p  RU8a  RD8a  TU10a  RU10a  RD10a  TD10a  11a
1     P      P      P      P     P     P      P      P      P      P    P
2   2.0    2.0    2.0    2.0   2.0   2.0   -1.0    0.0    4.0    3.0  0.0
3   4.0    2.0    3.0    5.0   1.0   5.0   -1.0    0.0    4.0    3.0  0.0
4   4.0    2.0    2.5    6.5   1.0   5.0   -1.0    0.0    4.0    3.0  0.0
5   4.0    2.0    2.5    6.5   1.0   5.0   -1.0    0.0    4.0    3.0  0.0


> Now initialized values of the states have changed to a new one. The next part will improve the policy based on policy iteration algorithm<br>
_*to get the results run again from the Vinit,Policy section._

In [120]:
policy_stable = True
iterations = 0
gamma = 1.0
proof = 0
table = np.vstack((get_state_names(states),get_policy_actions_or_values(Policy),get_policy_actions_or_values(Vinit)))
while(policy_stable): #policy is stable
    iterations += 1
#     print('Iterations:', iterations)
    check_action = 0
#     print(get_policy_actions_or_values(Policy))
#     print(get_policy_actions_or_values(Vinit))
    for s in states:
        if s.isendstate == False: #not end state
            old_action = Policy[s] #this is the starting policy
            uAct = len(set(s.actions)) #3
            if uAct == len(s.actions):  # no dual actions
                tsum = []
                for j in range(len(s.actions)):
                    # print("Singles",j)
                    tsum.append(s.probs[j]*(s.rewards[j]+gamma*Vinit[s.next_states[j]]))
                u = -1 #just to find max
                maximum = max(tsum)
                for ii in tsum:
                    u += 1
                    if ii == maximum:
                        break
                new_action = s.actions[u]
                if old_action == new_action:
                    check_action+=1
                else:
                    Policy[s] = new_action
                    Vinit,_ = Evaluation(Policy, Vinit, states)
            else: #dual actions
                ttsum = np.zeros(uAct)
                dual_sum = []
                for j in range(len(s.actions)):
                    if j<2:
                        dual_sum.append(s.probs[j]*(s.rewards[j]+gamma*Vinit[s.next_states[j]]))
                        d = 0
                        ttsum[d] = np.sum(dual_sum)
                    else:
                        d+=1
                        ttsum[d] = s.probs[j]*(s.rewards[j]+gamma*Vinit[s.next_states[j]])
                u = -1  #to find max
                maximum = max(ttsum)
                for ii in ttsum:
                    u += 1
                    if ii == maximum:
                        break
                new_action = s.actions[u]
                if old_action==new_action:
                    check_action += 1
                else:
                    Policy[s] = new_action
                    Vinit,_ = Evaluation(Policy,Vinit,states)
    table = np.vstack((table,get_policy_actions_or_values(Policy),get_policy_actions_or_values(Vinit)))
    if check_action == 10:
        proof +=1
    if proof == 2:
        policy_stable = False

table = pd.DataFrame(table)
# table = table[-1:]
print(table)

     0      1      2      3     4     5      6      7      8      9    10
0  RU8p  TU10p  RU10p  RD10p  RU8a  RD8a  TU10a  RU10a  RD10a  TD10a  11a
1     S      R      R      P     S     P      P      P      P      P    P
2   5.5    3.0    3.0    6.5   3.0   5.0   -1.0    0.0    4.0    3.0  0.0
3     S      R      R      P     S     P      P      P      P      P    P
4   5.5    3.0    3.0    6.5   3.0   5.0   -1.0    0.0    4.0    3.0  0.0
5     S      R      R      P     S     P      P      P      P      P    P
6   5.5    3.0    3.0    6.5   3.0   5.0   -1.0    0.0    4.0    3.0  0.0


The last iteration above proves that the policy does not improve beyond that iteration. That means the final policy and state values are optimal policy and value functions.